#### 解压数据集

In [1]:
import os
import zipfile

local_zip = 'E:/02Studying/03DeepLearning/dataset/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/horse-or-human')
local_zip = 'E:/02Studying/03DeepLearning/dataset/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/validation-horse-or-human')
zip_ref.close

<bound method ZipFile.close of <zipfile.ZipFile filename='E:/02Studying/03DeepLearning/dataset/validation-horse-or-human.zip' mode='r'>>

#### 划分数据集为训练集和验证集

In [2]:
train_horse_dir = os.path.join('E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/horse-or-human/horses')
train_hunman_dir = os.path.join('E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/horse-or-human/humans')

val_horse_dir = os.path.join('E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/validation-horse-or-human/horses')
val_human_dir = os.path.join('E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/validation-horse-or-human/humans')

#### 打印部分训练集和验证集

In [3]:
train_horse_names = os.listdir(train_horse_dir)
print(train_horse_names[:10])

train_human_names = os.listdir(train_hunman_dir)
print(train_human_names[:10])

val_horse_names = os.listdir(val_horse_dir)
print(val_horse_names[:10])

val_human_names = os.listdir(val_human_dir)
print(val_human_names[:10])

['horse01-0.png', 'horse01-1.png', 'horse01-2.png', 'horse01-3.png', 'horse01-4.png', 'horse01-5.png', 'horse01-6.png', 'horse01-7.png', 'horse01-8.png', 'horse01-9.png']
['human01-00.png', 'human01-01.png', 'human01-02.png', 'human01-03.png', 'human01-04.png', 'human01-05.png', 'human01-06.png', 'human01-07.png', 'human01-08.png', 'human01-09.png']
['horse1-000.png', 'horse1-105.png', 'horse1-122.png', 'horse1-127.png', 'horse1-170.png', 'horse1-204.png', 'horse1-224.png', 'horse1-241.png', 'horse1-264.png', 'horse1-276.png']
['valhuman01-00.png', 'valhuman01-01.png', 'valhuman01-02.png', 'valhuman01-03.png', 'valhuman01-04.png', 'valhuman01-05.png', 'valhuman01-06.png', 'valhuman01-07.png', 'valhuman01-08.png', 'valhuman01-09.png']


#### 输出训练集和验证集的数量

In [4]:
print('total training horse images: ', len(os.listdir(train_horse_dir)))
print('total training human images: ', len(os.listdir(train_hunman_dir)))
print('total val horse images: ', len(os.listdir(val_horse_dir)))
print('total val human images: ', len(os.listdir(val_human_dir)))

total training horse images:  500
total training human images:  527
total val horse images:  128
total val human images:  128


#### 构建模型

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#### 输出搭建的模型

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

#### 设置模型的优化器

In [8]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

#### 生成训练集和验证集

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    'E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/horse-or-human',
    target_size=(150, 150),
    batch_size=128,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    'E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/validation-horse-or-human',
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


#### 开始训练

In [10]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = val_generator,
      validation_steps=8
      )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
8/8 [==============================] - 4s 502ms/step - loss: 1.8029 - acc: 0.4980 - val_loss: 0.5430 - val_acc: 0.6992
Epoch 2/15
8/8 [==============================] - 4s 476ms/step - loss: 0.6278 - acc: 0.6452 - val_loss: 0.5152 - val_acc: 0.7539
Epoch 3/15
8/8 [==============================] - 4s 474ms/step - loss: 0.9082 - acc: 0.7486 - val_loss: 1.1273 - val_acc: 0.6172
Epoch 4/15
8/8 [==============================] - 4s 476ms/step - loss: 0.3877 - acc: 0.8242 - val_loss: 0.7536 - val_acc: 0.8242
Epoch 5/15
8/8 [==============================] - 4s 470ms/step - loss: 0.1815 - acc: 0.9277 - val_loss: 1.3637 - val_acc: 0.7461
Epoch 6/15
8/8 [==============================] - 4s 473ms/step - loss: 0.0977 - acc: 0.9588 - val_loss: 0.9615 - val_acc: 0.8398
Epoch 7/15
8/8 [==============================] - 4s 474ms/step - loss: 0.0511 - acc: 0.9822 - val_loss: 0.4929 - val_acc: 0.9023
Epoch 8/15
8/8

#### 测试训练的模型效果

In [11]:
import numpy as np 
import cv2
from keras.preprocessing import image

image_dir = 'E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/test-horse-or-human'

image_names = os.listdir(image_dir)
sorted_image_names = sorted(image_names)
for name in sorted_image_names:
    image_name = os.path.join(image_dir, name)
    image = cv2.imread(image_name)

    x = cv2.resize(image, (150, 150))

# for fn in uploaded.keys():

#     path = '/content' + fn
#     img = image.load_img(path, target_size=(150,150))
#     x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=20)
    print(classes[0])
    if classes[0]>0.5:
        print(image_name + " is a human")
    else:
        print(image_name + " is a horse")


[1.]
E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/test-horse-or-human\hor1.jpg is a human
[0.]
E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/test-horse-or-human\hor2.jpg is a horse
[1.]
E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/test-horse-or-human\hor3.jpg is a human
[1.]
E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/test-horse-or-human\hor4.jpg is a human
[1.]
E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/test-horse-or-human\hum1.jpg is a human
[1.]
E:/02Studying/03DeepLearning/dataset/jupyter/04horse_human/test-horse-or-human\hum2.jpg is a human


#### 清除内存

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)